# ATTN: This script uses Google translate to detect job description language. Google translate will limit requests and take a very long time. Only run this script if redoing language detection.

# Read from scrapped data

In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

#### Read paths

In [3]:
glob_paths = list(set(glob.glob(f'{scraped_data}Coding Material/*Folder/*/Job ID -*- Codebook (Automating Equity).xlsx')))


In [4]:
# 244 xlsx files
len(glob_paths)


244

#### Use paths to open files, fix keywords, and drop unneeded columns

In [5]:
%%time
# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

# Fix list catches all incorrect/faculty keyword search terms
fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    try:
        df_temp = pd.read_excel(glob_path).reset_index(drop=True)
    except ValueError:
        fix_list.append(glob_path)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = df_temp.reset_index(drop=True)
        df_temp = df_temp.drop(columns=cols, axis='columns', errors='ignore')
        df_temp = df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            errors='ignore',
        )

        appended_data.append(df_temp.reset_index(drop=True))

# Concatonate list of dfs into one large df_manual
df_manual = pd.concat(appended_data, axis='index').reset_index(drop=True)

# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_raw.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw.csv', index=False)


CPU times: user 5.59 s, sys: 118 ms, total: 5.71 s
Wall time: 5.91 s


In [6]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


In [7]:
# List of dfs, len = 244
len(appended_data)


244

In [8]:
# Concatonate list of dfs into one large df_manual
df_manual = pd.concat(appended_data, axis='index').reset_index(drop=True)


In [9]:
# len = 12400
len(df_manual)


12400

In [10]:
# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_raw.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw.csv', index=False)


# Drop duplicated and missing data

### START HERE IF SOURCING FROM df_manual_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [11]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [12]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8

In [13]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw.pkl').reset_index(drop=True)


In [14]:
# len = 12400
len(df_manual)


12400

In [15]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12400 entries, 0 to 12399
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Job ID           12400 non-null  object 
 1   Sentence         12396 non-null  object 
 2   Warmth           12398 non-null  float64
 3   Competence       12400 non-null  int64  
 4   Task_Mentioned   12398 non-null  float64
 5   Task_Warmth      12398 non-null  float64
 6   Task_Competence  12398 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 678.2+ KB


In [16]:
# Clean columns
df_manual.columns = df_manual.columns.to_series().progress_apply(lambda x: str(x).strip())

progress-bar:   0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
# Remove columns 'Task_Mentioned', 'Task_Warmth', 'Task_Competence'
df_manual = df_manual.drop(
    columns=['Task_Mentioned', 'Task_Warmth', 'Task_Competence'],
    axis='columns',
    errors='ignore'
)

In [18]:
df_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12400 entries, 0 to 12399
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Job ID      12400 non-null  object 
 1   Sentence    12396 non-null  object 
 2   Warmth      12398 non-null  float64
 3   Competence  12400 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 387.6+ KB


In [19]:
# Missing values: Sentence = 4, Warmth = 2, Competence = 0
df_manual.isna().sum()

Job ID        0
Sentence      4
Warmth        2
Competence    0
dtype: int64

In [20]:
# Drop NA
df_manual = df_manual.dropna(axis='index', how='all')
df_manual = df_manual.dropna(axis='columns', how='all')
df_manual = df_manual.dropna(
    subset = ['Sentence', 'Warmth', 'Competence'],
)

In [21]:
# No na values
df_manual.isna().sum()

Job ID        0
Sentence      0
Warmth        0
Competence    0
dtype: int64

In [22]:
df_manual.columns

Index(['Job ID', 'Sentence', 'Warmth', 'Competence'], dtype='object')

In [23]:
# Convert Warmth and Competence to int
int_cols = [
    'Warmth',
    'Competence',
]

for col in int_cols:
    df_manual[col] = df_manual[col].astype(np.int64, errors='ignore')
    print(f'{col} converted to int.' if all(df_manual[col].progress_apply(lambda x: isinstance(x, int))) else f'{col} NOT converted to int.')
    print(f'{col} value counts:\n{df_manual[col].value_counts()}')


progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

Warmth converted to int.
Warmth value counts:
0    9568
1    2826
Name: Warmth, dtype: int64


progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

Competence converted to int.
Competence value counts:
0    7330
1    5064
Name: Competence, dtype: int64


In [24]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12394 entries, 0 to 12399
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Job ID      12394 non-null  object
 1   Sentence    12394 non-null  object
 2   Warmth      12394 non-null  int64 
 3   Competence  12394 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 484.1+ KB


In [25]:
%%time
# Conver Job ID and Sentence to str
str_cols = [
    'Job ID',
    'Sentence',
]

for col in str_cols:
    df_manual[col] = df_manual[col].astype(str, errors='ignore').progress_apply(lambda x: x.strip().replace('[', '').replace(']', ''))
    print(f'{col} converted to str.' if all(df_manual[col].progress_apply(lambda x: isinstance(x, str))) else f'{col} NOT converted to str.')


progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

Job ID converted to str.


progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/12394 [00:00<?, ?it/s]

Sentence converted to str.
CPU times: user 65.1 ms, sys: 5.85 ms, total: 70.9 ms
Wall time: 96.3 ms


In [26]:
# Remove any rows with missing 'Job ID'
df_manual = df_manual.drop(
    df_manual[
        (df_manual['Sentence'].isin(nan_list)) | 
        (df_manual['Sentence'].isnull()) | 
        (df_manual['Sentence'].isna())
    ].index, 
    axis='index',
    errors='ignore'
)

In [27]:
df_manual = df_manual.drop_duplicates(subset=['Sentence'], keep='first', ignore_index=True)

In [28]:
# len = 5681
len(df_manual)

5681

In [29]:
df_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Job ID      5681 non-null   object
 1   Sentence    5681 non-null   object
 2   Warmth      5681 non-null   int64 
 3   Competence  5681 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 177.7+ KB


In [30]:
# Rename Sentence to 'Job Description spacy_sentencized'
df_manual = df_manual.rename(
    columns = {
        'Sentence': 'Job Description spacy_sentencized'
    },
    errors='ignore'
)

In [31]:
df_manual.columns


Index(['Job ID', 'Job Description spacy_sentencized', 'Warmth', 'Competence'], dtype='object')

In [32]:
# Drop NA
df_manual = df_manual.dropna(axis='index', how='all')
df_manual = df_manual.dropna(axis='columns', how='all')
df_manual = df_manual.dropna(
    subset = ['Job Description spacy_sentencized', 'Warmth', 'Competence'],
)


In [33]:
# len = 5681
len(df_manual)


5681

In [34]:
# len = 126
len(df_manual.groupby(['Job ID'])['Job ID'].unique())


126

In [35]:
# Drop duplicates on subset of 'Job Description'
df_manual = df_manual.drop_duplicates(subset=['Job Description spacy_sentencized'], keep='first', ignore_index=True)


In [36]:
# len = 5681
len(df_manual)


5681

In [37]:
# Remove any rows with missing 'Job ID'
df_manual = df_manual.drop(
    df_manual[
        (df_manual['Job ID'].isin(nan_list)) | 
        (df_manual['Job ID'].isnull()) | 
        (df_manual['Job ID'].isna())
    ].index, 
    axis='index',
    errors='ignore'
)


In [38]:
# Remove any rows with missing 'Job Description'
df_manual = df_manual.drop(
    df_manual[
        (df_manual['Job Description spacy_sentencized'].isin(nan_list)) | 
        (df_manual['Job Description spacy_sentencized'].isnull()) | 
        (df_manual['Job Description spacy_sentencized'].isna())
    ].index, 
    axis='index',
    errors='ignore'
)


In [39]:
# len = 5681
len(df_manual)


5681

In [40]:
# Save df_manual to file
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_dropped.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_dropped.csv', index=False)


# Add English and Dutch language requirement columns

### START HERE IF SOURCING FROM df_manual_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [41]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [42]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8

In [43]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw_dropped.pkl').reset_index(drop=True)


In [44]:
# 5681
len(df_manual)


5681

In [45]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 4 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Job ID                             5681 non-null   object
 1   Job Description spacy_sentencized  5681 non-null   object
 2   Warmth                             5681 non-null   int64 
 3   Competence                         5681 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 177.7+ KB


In [46]:
%%time
# Add language requirement column
# Use regex to find language requirement
dutch_requirement_pattern = r'[Ll]anguage: [Dd]utch|[Dd]utch [Pp]referred|[Dd]utch [Re]quired|[Dd]utch [Ll]anguage|[Pp]roficient in [Dd]utch|[Ss]peak [Dd]utch|[Kk]now [Dd]utch'
english_requirement_pattern = r'[Ll]anguage: [Ee]nglish|[Ee]nglish [Pp]referred|[Ee]nglish [Re]quired|[Ee]nglish [Ll]anguage|[Pp]roficient in [Ee]nglish|[Ss]peak [Ee]nglish|[Kk]now [Ee]nglish'

lang_requirements = {
    'Dutch Requirement': dutch_requirement_pattern, 'English Requirement': english_requirement_pattern
}

for lang_req, lang_req_pattern in lang_requirements.items():

    if lang_req in df_manual.columns:
        df_manual = df_manual.drop(columns=[lang_req])
    df_manual[lang_req] = np.where(
        df_manual['Job Description spacy_sentencized'].str.contains(lang_req_pattern),
        1,
        0,
    )

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_english_requirement.csv', index=False)


CPU times: user 62 ms, sys: 2.33 ms, total: 64.4 ms
Wall time: 63.6 ms


In [47]:
# Dutch Yes = 6, English Yes = 8
df_manual[['Dutch Requirement', 'English Requirement']].value_counts()


Dutch Requirement  English Requirement
0                  0                      5667
                   1                         8
1                  0                         6
dtype: int64

In [48]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_raw_language_requirement.csv', index=False)


# Add data from Sectors dataframe (see CBS directory under scrapped_data directory) and Categorical data


### START HERE IF SOURCING FROM df_manual_RAW_LANGUAGE_REQUIREMENT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [49]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [50]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8

In [51]:
def df_gender_age_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            counts = df[iv].value_counts()
            percentages = df[iv].value_counts(normalize=True).mul(100).round(1).astype(float)
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            print(f'{iv} Counts:\n{counts}')
            print('-'*20)
            print(f'{iv} Percentages:\n{percentages}')

            with contextlib.suppress(Exception):
                mean = df[f"{iv}"].mean().round(2).astype(float)
                sd = df[f"{iv}"].std().round(2).astype(float)
                print('-'*20)
                print(f'{iv} Mean: {mean}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {sd}')

        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [52]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_raw_language_requirement.pkl').reset_index(drop=True)


In [53]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Job ID                             5681 non-null   object
 1   Job Description spacy_sentencized  5681 non-null   object
 2   Warmth                             5681 non-null   int64 
 3   Competence                         5681 non-null   int64 
 4   Dutch Requirement                  5681 non-null   int64 
 5   English Requirement                5681 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 266.4+ KB


In [54]:
df_manual['Job ID'] = df_manual['Job ID'].progress_apply(lambda x: str(x).lower().strip())


progress-bar:   0%|          | 0/5681 [00:00<?, ?it/s]

In [55]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_including_sector_genage_data.pkl').reset_index(drop=True)


In [56]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17599 entries, 0 to 17598
Data columns (total 60 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17599 non-null  object  
 1   Platform                             17599 non-null  category
 2   Job ID                               17599 non-null  object  
 3   Job Title                            17599 non-null  object  
 4   Company Name                         17597 non-null  object  
 5   Location                             17599 non-null  object  
 6   Job Description                      17599 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17017 non-null  object  
 9   Company URL                          15959 non-null  object  
 10  Job URL                              17599 non-null  object  
 11  Job Age        

In [57]:
df_jobs['Job ID'] = df_jobs['Job ID'].progress_apply(lambda x: str(x).lower().strip())


progress-bar:   0%|          | 0/17599 [00:00<?, ?it/s]

In [58]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership', 'Language', 'Dutch Requirement', 'English Requirement', 'Sector Code', 'Sector', 'Keywords Count', '% per Sector', '% per Social Category', '% per Workforce', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce', 'Gender_Female', 'Gender_Male', 'Gender_Mixed', 'Age

In [59]:
df_jobs = df_jobs.drop(
    columns = [
        'Job Description', 'Rating',
        'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
        'Collection Date', 'Data Row', 'Tracking ID', 'Job Date',
        'Type of ownership', 'Language', 'Dutch Requirement', 'English Requirement', 
    ],
    errors='ignore'
)

In [60]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Employment Type', 'Industry', 'Type of ownership', 'Sector Code', 'Sector', 'Keywords Count', '% per Sector', '% per Social Category', '% per Workforce', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce', 'Gender_Female', 'Gender_Male', 'Gender_Mixed', 'Age_Mixed', 'Age_Older', 'Age_Younger', 'Gender_Num', 'Age_Num', 'Platform_Num', 'Platform_LinkedIn', 'Platform_Indeed', 'Platform_Glassdoor'], dtype='object')

In [61]:
# Add sector and categorical data from df_jobs
df_manual = df_manual.merge(df_jobs, on='Job ID', how='inner')


In [62]:
# len = 5298
len(df_manual)


5365

In [63]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5365 entries, 0 to 5364
Data columns (total 52 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Job ID                               5365 non-null   object  
 1   Job Description spacy_sentencized    5365 non-null   object  
 2   Warmth                               5365 non-null   int64   
 3   Competence                           5365 non-null   int64   
 4   Dutch Requirement                    5365 non-null   int64   
 5   English Requirement                  5365 non-null   int64   
 6   Search Keyword                       5365 non-null   object  
 7   Platform                             5365 non-null   category
 8   Job Title                            5365 non-null   object  
 9   Company Name                         5365 non-null   object  
 10  Location                             5365 non-null   object  
 11  Employment Type  

In [64]:
df_manual.head()

,Job ID,Job Description spacy_sentencized,Warmth,Competence,Dutch Requirement,English Requirement,Search Keyword,Platform,Job Title,Company Name,Location,Employment Type,Industry,Type of ownership,Sector Code,Sector,Keywords Count,% per Sector,% per Social Category,% per Workforce,Gender_Female_n,Gender_Female_% per Sector,Gender_Female_% per Social Category,Gender_Female_% per Workforce,Gender_Male_n,Gender_Male_% per Sector,Gender_Male_% per Social Category,Gender_Male_% per Workforce,Gender,Age_Older_n,Age_Older_% per Sector,Age_Older_% per Social Category,Age_Older_% per Workforce,Age_Younger_n,Age_Younger_% per Sector,Age_Younger_% per Social Category,Age_Younger_% per Workforce,Age,Sector_n,% Sector per Workforce,Gender_Female,Gender_Male,Gender_Mixed,Age_Mixed,Age_Older,Age_Younger,Gender_Num,Age_Num,Platform_Num,Platform_LinkedIn,Platform_Indeed,Platform_Glassdoor
0,3768944208,We’re growing our Sales team for the EMEA Mark...,1,0,0,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,NaN,Ondernemingssoftware en netwerkoplossingen,Privébedrijf,Q,Health and social work activities,11.00,0.01,0.11,0.00,1208.00,0.84,0.10,0.05,224.00,0.16,0.02,0.01,Female,661.00,0.46,0.06,0.03,770.00,0.54,0.05,0.03,Mixed Age,1433.00,0.06,1,0,0,1,0,0,0,1,2,0,0,1
1,3768944208,Are you looking for an adventure – one with bi...,0,1,0,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,NaN,Ondernemingssoftware en netwerkoplossingen,Privébedrijf,Q,Health and social work activities,11.00,0.01,0.11,0.00,1208.00,0.84,0.10,0.05,224.00,0.16,0.02,0.01,Female,661.00,0.46,0.06,0.03,770.00,0.54,0.05,0.03,Mixed Age,1433.00,0.06,1,0,0,1,0,0,0,1,2,0,0,1
2,3768944208,Do you have experience closing B2B Saa,0,0,0,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,NaN,Ondernemingssoftware en netwerkoplossingen,Privébedrijf,Q,Health and social work activities,11.00,0.01,0.11,0.00,1208.00,0.84,0.10,0.05,224.00,0.16,0.02,0.01,Female,661.00,0.46,0.06,0.03,770.00,0.54,0.05,0.03,Mixed Age,1433.00,0.06,1,0,0,1,0,0,0,1,2,0,0,1
3,3768944208,S deals in the mid-market and enterprise space?,0,0,0,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,NaN,Ondernemingssoftware en netwerkoplossingen,Privébedrijf,Q,Health and social work activities,11.00,0.01,0.11,0.00,1208.00,0.84,0.10,0.05,224.00,0.16,0.02,0.01,Female,661.00,0.46,0.06,0.03,770.00,0.54,0.05,0.03,Mixed Age,1433.00,0.06,1,0,0,1,0,0,0,1,2,0,0,1
4,3768944208,Then you might be ready to become part of the ...,0,1,0,0,social work activity,Glassdoor,Inbound Marketer,Happeo,Amsterdam,NaN,Ondernemingssoftware en netwerkoplossingen,Privébedrijf,Q,Health and social work activities,11.00,0.01,0.11,0.00,1208.00,0.84,0.10,0.05,224.00,0.16,0.02,0.01,Female,661.00,0.46,0.06,0.03,770.00,0.54,0.05,0.03,Mixed Age,1433.00,0.06,1,0,0,1,0,0,0,1,2,0,0,1


#### Check if there is any missing sector data in the merged dataframe

In [65]:
df_manual['Sector'].isna().sum()

0

In [66]:
if df_manual['Sector'].isna().sum() != 0:
    print('Some search keywords did not match a sector. Fixing')
    print(set(df_manual['Search Keyword'].loc[df_manual['Sector'].isna()].to_list()))
    print(len(df_manual['Search Keyword'].loc[df_manual['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_manual = fix_keywords(df_manual)
    print(set(df_manual['Search Keyword'].loc[df_manual['Sector'].isna()].to_list()))
    print(len(df_manual['Search Keyword'].loc[df_manual['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [67]:
# Manual Job Ad info, len = 113
df_gender_age_info(df_manual.groupby(['Job ID']).first())



DF INFO:

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, 3768944208 to pj_a4ac3e531abef752
Data columns (total 51 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Job Description spacy_sentencized    114 non-null    object  
 1   Warmth                               114 non-null    int64   
 2   Competence                           114 non-null    int64   
 3   Dutch Requirement                    114 non-null    int64   
 4   English Requirement                  114 non-null    int64   
 5   Search Keyword                       114 non-null    object  
 6   Platform                             114 non-null    category
 7   Job Title                            114 non-null    object  
 8   Company Name                         114 non-null    object  
 9   Location                             114 non-null    object  
 10  Employment Type                      109 non-null    ob

In [68]:
# Manual Job Sentence info
df_gender_age_info(df_manual)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5365 entries, 0 to 5364
Data columns (total 52 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Job ID                               5365 non-null   object  
 1   Job Description spacy_sentencized    5365 non-null   object  
 2   Warmth                               5365 non-null   int64   
 3   Competence                           5365 non-null   int64   
 4   Dutch Requirement                    5365 non-null   int64   
 5   English Requirement                  5365 non-null   int64   
 6   Search Keyword                       5365 non-null   object  
 7   Platform                             5365 non-null   category
 8   Job Title                            5365 non-null   object  
 9   Company Name                         5365 non-null   object  
 10  Location                             5365 non-null   object  
 11  Employ

In [69]:
# Manual Job Sentence info
df_gender_age_info(df_manual, ivs_all=['Warmth', 'Competence'])



DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5365 entries, 0 to 5364
Data columns (total 52 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Job ID                               5365 non-null   object  
 1   Job Description spacy_sentencized    5365 non-null   object  
 2   Warmth                               5365 non-null   int64   
 3   Competence                           5365 non-null   int64   
 4   Dutch Requirement                    5365 non-null   int64   
 5   English Requirement                  5365 non-null   int64   
 6   Search Keyword                       5365 non-null   object  
 7   Platform                             5365 non-null   category
 8   Job Title                            5365 non-null   object  
 9   Company Name                         5365 non-null   object  
 10  Location                             5365 non-null   object  
 11  Employ

In [70]:
if df_manual['Sector'].isna().sum() == 0:
    assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
    df_manual.to_pickle(f'{df_save_dir}df_manual_including_sector_genage_data.pkl')
    df_manual.to_csv(f'{df_save_dir}df_manual_including_sector_genage_data.csv', index=False)
else:
    print(f"MISSING SECTOR DATA: COUNT {df_manual['Sector'].isna().sum()}")

# ATTN: This script should be run AFTER spacy sentence splitting is completed.


# Use spacy to tokenize sentences


### START HERE IF SOURCING FROM df_manual_SENTENCIZED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [71]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [72]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8

In [73]:
def get_word_num_and_frequency(row, text_col):

    with open(f'{data_dir}punctuations.txt', 'rb') as f:
        custom_punct_chars = pickle.load(f)
    row['Job Description num_words'] = len(str(row[text_col]).split())
    row['Job Description num_unique_words'] = len(set(str(row[text_col]).split()))
    row['Job Description num_chars'] = len(str(row[text_col]))
    row['Job Description num_chars_no_whitespact_and_punt'] = len(
        [
            c.translate({ord(s): None for s in string.whitespace})
            for c in str(row[text_col])
            if c not in custom_punct_chars and c not in string.punctuation
        ]
    )
    row['Job Description num_punctuations'] = len(
        [
            c
            for c in str(row[text_col])
            if c in custom_punct_chars and c in string.punctuation
        ]
    )

    return row


In [74]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_including_sector_genage_data.pkl').reset_index(drop=True)


In [75]:
df_manual['Job Description spacy_sentencized_lower'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: job_sentence.strip().lower()
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

In [76]:
df_manual[['Job Description spacy_sentencized', 'Job Description spacy_sentencized_lower']].head()


,Job Description spacy_sentencized,Job Description spacy_sentencized_lower
0,We’re growing our Sales team for the EMEA Mark...,we’re growing our sales team for the emea mark...
1,Are you looking for an adventure – one with bi...,are you looking for an adventure – one with bi...
2,Do you have experience closing B2B Saa,do you have experience closing b2b saa
3,S deals in the mid-market and enterprise space?,s deals in the mid-market and enterprise space?
4,Then you might be ready to become part of the ...,then you might be ready to become part of the ...


In [77]:
%%time
# Spacy tokenize
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_manual['Job Description spacy_tokenized'] = df_manual[
    'Job Description spacy_sentencized'
].progress_apply(
    lambda job_sentence: [
        str(token.text.strip().lower())
        for token in nlp.tokenizer(job_sentence)
        if len(token) != 0
        and not token.is_space
        and not token.is_stop
        and not token.is_punct
        and not token.is_bracket
        and not token.like_email
        and token.text not in custom_punct_chars
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy.csv', index=False)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 470 ms, sys: 15.7 ms, total: 486 ms
Wall time: 492 ms


In [78]:
df_manual['Job Description spacy_sentencized_cleaned'] = df_manual['Job Description spacy_tokenized'].str.join(' ')


In [79]:
%%time
# Get sentence word frequencies
df_manual = df_manual.progress_apply(
    lambda row: get_word_num_and_frequency(
        row=row, text_col='Job Description spacy_sentencized'
    ), 
    axis='columns',
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 5.15 s, sys: 167 ms, total: 5.31 s
Wall time: 5.49 s


In [81]:
df_manual[
    [
        'Job Description spacy_sentencized',
        'Job Description num_words', 'Job Description num_unique_words',
        'Job Description num_chars', 'Job Description num_chars_no_whitespact_and_punt'
    ]
].head()

,Job Description spacy_sentencized,Job Description num_words,Job Description num_unique_words,Job Description num_chars,Job Description num_chars_no_whitespact_and_punt
0,We’re growing our Sales team for the EMEA Mark...,19,19,91,90
1,Are you looking for an adventure – one with bi...,11,11,63,62
2,Do you have experience closing B2B Saa,7,7,38,38
3,S deals in the mid-market and enterprise space?,8,8,47,45
4,Then you might be ready to become part of the ...,17,17,91,89


In [82]:
df_manual.columns


Index(['Job ID', 'Job Description spacy_sentencized', 'Warmth', 'Competence', 'Dutch Requirement', 'English Requirement', 'Search Keyword', 'Platform', 'Job Title', 'Company Name', 'Location', 'Employment Type', 'Industry', 'Type of ownership', 'Sector Code', 'Sector', 'Keywords Count', '% per Sector', '% per Social Category', '% per Workforce', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce', 'Gender_Female', 'Gender_Male', 'Gender_Mixed', 'Age_Mixed', 'Age_Older', 'Age_Younger', 'Gender_Num', 'Age_Num', 'Platform_Num', 'Platform_Linke

In [83]:
# Job Sentence info
df_gender_age_info(df_manual)


DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 60 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company 

In [84]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy.csv', index=False)


# Use NLTK to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [85]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [86]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [87]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy.pkl').reset_index(drop=True)


In [88]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 60 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [89]:
%%time
# Tokenize with NLTK
# stop_words = set(stopwords.words('english'))
# punctuations = list(string.punctuation)
# lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()

df_manual['Job Description nltk_tokenized'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk.csv', index=False)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 4.96 s, sys: 1.71 s, total: 6.67 s
Wall time: 8.82 s


In [90]:
df_manual['Job Description nltk_tokenized'].head()


0    [’, growing, sales, team, emea, market, want, ...
1    [looking, adventure, –, one, big, responsibility]
2                      [experience, closing, b2b, saa]
3               [deals, mid-market, enterprise, space]
4    [might, ready, become, part, team, drives, hap...
Name: Job Description nltk_tokenized, dtype: object

In [91]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 61 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [92]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk.csv', index=False)


# Use gensim to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [93]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [94]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [95]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk.pkl').reset_index(drop=True)


In [96]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 61 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [97]:
%%time
df_manual['Job Description gensim_tokenized'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.csv', index=False)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 486 ms, sys: 19.7 ms, total: 505 ms
Wall time: 559 ms


In [98]:
df_manual['Job Description gensim_tokenized'].head()


0    [we’r, grow, sale, team, emea, market, want, j...
1                       [look, adventur, big, respons]
2                                 [experi, close, saa]
3                [deal, mid, market, enterpris, space]
4    [readi, team, drive, happeo, sky, high, worldwid]
Name: Job Description gensim_tokenized, dtype: object

In [99]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 62 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [100]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.csv', index=False)


# Use BERT to tokenize sentences


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [101]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [102]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [103]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim.pkl').reset_index(drop=True)


In [104]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 62 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [105]:
%%time
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name).to(device)

df_manual['Job Description bert_encodings'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: bert_tokenizer(
        str(sentence), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    )
)

df_manual['Job Description bert_tokenized'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: bert_tokenizer.tokenize(str(sentence))
)

df_manual['Job Description bert_tokenized_to_id'] = df_manual['Job Description bert_tokenized'].progress_apply(
    lambda sentence: bert_tokenizer.convert_tokens_to_ids(str(sentence))
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.csv', index=False)


Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 3.59 s, sys: 840 ms, total: 4.43 s
Wall time: 5.6 s


In [106]:
df_manual['Job Description bert_encodings'].head()


0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: Job Description bert_encodings, dtype: object

In [107]:
df_manual['Job Description bert_tokenized'].head()


0    [we, ’, re, growing, our, sales, team, for, th...
1    [are, you, looking, for, an, adventure, –, one...
2    [do, you, have, experience, closing, b, ##2, #...
3    [s, deals, in, the, mid, -, market, and, enter...
4    [then, you, might, be, ready, to, become, part...
Name: Job Description bert_tokenized, dtype: object

In [108]:
df_manual['Job Description bert_tokenized_to_id'].head()

0    100
1    100
2    100
3    100
4    100
Name: Job Description bert_tokenized_to_id, dtype: int64

In [109]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.csv', index=False)


# ATTN: This script should be run AFTER all tokenization (spacy, nltk, gensim, and BERT) completed.


# Use spacy to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TOKENIZED_SPACY_NLTK_GENSIM_BERT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [110]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [111]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [112]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tokenized_spacy_nltk_gensim_bert.pkl').reset_index(drop=True)


In [113]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 65 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [114]:
%%time
# Load customer characters
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

# POS tagging
df_manual['Job Description spacy_token_tags'] = df_manual[
    'Job Description spacy_sentencized'
].progress_apply(
    lambda job_sentence: [
        (token.text.strip().lower(), token.tag_) for token in nlp(job_sentence)
    ]
)

# Lemmatization
df_manual['Job Description spacy_lemmas'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        token.lemma_.strip().lower()
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)

# Stemming
df_manual['Job Description spacy_stems'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        stemmer.stem(token.text.strip().lower())
        for token in nlp(job_sentence)
        if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.csv', index=False)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 1min 19s, sys: 1.58 s, total: 1min 21s
Wall time: 1min 32s


In [115]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 68 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [116]:
df_manual[
    [
        'Job Description spacy_token_tags',
        'Job Description spacy_lemmas',
        'Job Description spacy_stems'
    ]
].head()


,Job Description spacy_token_tags,Job Description spacy_lemmas,Job Description spacy_stems
0,"[(we, PRP), (’re, VBP), (growing, VBG), (our, ...","[grow, sale, team, emea, market, want, journey]","[grow, sale, team, emea, market, want, journey]"
1,"[(are, VBP), (you, PRP), (looking, VBG), (for,...","[look, adventure, big, responsibility]","[look, adventur, big, respons]"
2,"[(do, VBP), (you, PRP), (have, VB), (experienc...","[experience, close, b2b, saa]","[experi, close, b2b, saa]"
3,"[(s, NN), (deals, VBZ), (in, IN), (the, DT), (...","[s, deal, mid, market, enterprise, space]","[s, deal, mid, market, enterpris, space]"
4,"[(then, RB), (you, PRP), (might, MD), (be, VB)...","[ready, team, drive, happeo, sky, high, worldw...","[readi, team, drive, happeo, sky, high, worldwid]"


In [117]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.csv', index=False)


# Use NLTK to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [118]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [119]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [120]:
def get_wordnet_pos(token):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


In [121]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy.pkl').reset_index(drop=True)


In [122]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 68 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [123]:
%%time
# POS tagging
df_manual['Job Description nltk_token_tags'] = df_manual['Job Description spacy_tokenized'].progress_apply(
    lambda token: pos_tag(token)
)

# Lemmatization
df_manual['Job Description nltk_lemmas'] = df_manual['Job Description spacy_tokenized'].progress_apply(
    lambda tokens: [
        lemmatizer.lemmatize(
            token, get_wordnet_pos(
                unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            )
        )
        for token in tokens
    ]
)

# Stemming
df_manual['Job Description nltk_stems'] = df_manual['Job Description spacy_tokenized'].progress_apply(
    lambda tokens: [
        stemmer.stem(
            unicodedata.normalize('NFKD', str(token.strip().lower())).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        )
        for token in tokens
    ]
)

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.csv', index=False)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 8.56 s, sys: 1.55 s, total: 10.1 s
Wall time: 11.3 s


In [124]:
df_manual.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5365 entries, 0 to 5364
Data columns (total 71 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Job ID                                            5365 non-null   object 
 1   Job Description spacy_sentencized                 5365 non-null   object 
 2   Warmth                                            5365 non-null   int64  
 3   Competence                                        5365 non-null   int64  
 4   Dutch Requirement                                 5365 non-null   int64  
 5   English Requirement                               5365 non-null   int64  
 6   Search Keyword                                    5365 non-null   object 
 7   Platform                                          5365 non-null   object 
 8   Job Title                                         5365 non-null   object 
 9   Company Name       

In [125]:
df_manual[['Job Description nltk_token_tags', 'Job Description nltk_lemmas', 'Job Description nltk_stems']].head()


,Job Description nltk_token_tags,Job Description nltk_lemmas,Job Description nltk_stems
0,"[(growing, VBG), (sales, NNS), (team, NN), (em...","[grow, sale, team, emea, market, want, journey]","[grow, sale, team, emea, market, want, journey]"
1,"[(looking, VBG), (adventure, NN), (big, JJ), (...","[look, adventure, big, responsibility]","[look, adventur, big, respons]"
2,"[(experience, NN), (closing, NN), (b2b, NN), (...","[experience, closing, b2b, saa]","[experi, close, b2b, saa]"
3,"[(s, JJ), (deals, NNS), (mid, VBP), (market, N...","[s, deal, mid, market, enterprise, space]","[s, deal, mid, market, enterpris, space]"
4,"[(ready, JJ), (team, NN), (drives, NNS), (happ...","[ready, team, drive, happeo, sky, high, worldw...","[readi, team, drive, happeo, sky, high, worldwid]"


In [126]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.csv', index=False)


# Use BERT to create Parts-Of-Speech (POS) tags, lemmas, and stems


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [127]:
# import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
# import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
# from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

# mod = sys.modules[__name__]

# code_dir = None
# code_dir_name = 'Code'
# unwanted_subdir_name = 'Analysis'

# for _ in range(5):

#     parent_path = str(Path.cwd().parents[_]).split('/')[-1]

#     if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

#         code_dir = str(Path.cwd().parents[_])

#         if code_dir is not None:
#             break

# sys.path.append(code_dir)
# # %load_ext autoreload
# # %autoreload 2


In [128]:
# from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [129]:
# df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)


In [130]:
# %%time
# bert_pos_model_name = 'QCRI/bert-base-multilingual-cased-pos-english'
# bert_pos_model = AutoModelForTokenClassification.from_pretrained(bert_pos_model_name).to(device)
# bert_pos_tagger = TokenClassificationPipeline(model=bert_pos_model, tokenizer=bert_tokenizer).to(device)

# df_manual['Job Description bert_token_tags_with_scores'] = df_manual['Job Description spacy_sentencized'].progress_apply(
#     lambda sentence: [
#         (bert_pos_tag['word'], bert_pos_tag['entity'], bert_pos_tag['score'])
#         for i in range(len(sentence.split()))
#         for bert_pos_tag in bert_pos_tagger(sentence)
#     ]
# )

# df_manual['Job Description bert_token_tags'] = df_manual['Job Description bert_token_tags_with_scores'].progress_apply(
#     lambda tag_list: [
#         [(tag_list[i][0], tag_list[i][1])]
#         for tag_tuple in tag_list
#         for i in range(len(tag_list))
#     ]
# )


# assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
# df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.pkl')
# df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.csv', index=False)


In [131]:
# df_manual['Job Description bert_token_tags'].head()

In [132]:
# assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
# df_manual.to_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.pkl')
# df_manual.to_csv(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk_bert.csv', index=False)


# ATTN: This script should be run AFTER all POS tagging, lemmatization, and stemming (spacy and nltk) completed.
# If BERT POS tagging was done, change pkl file loading


### START HERE IF SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK
### IF BERT POS TAGGING WAS DONE, SOURCING FROM df_manual_TAGS_LEMMAS_STEMS_SPACY_NLTK_BERT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Use spacy to create bi and trigrams


In [133]:
import os
import sys
import importlib
from pathlib import Path
import numpy as np

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [134]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [135]:
def spacy_make_ngrams(sentence, matcher, gram_type):

    doc = nlp(sentence)
    matches = matcher(doc)
    matches_list = []

    for idx in range(len(matches)):
        for match_id, start, end in matches:
            if nlp.vocab.strings[match_id].split('_')[0] == gram_type:
                match = doc[matches[idx][1]: matches[idx][2]].text
                matches_list.append(match.lower())
    
    return list(set(matches_list))


In [136]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_tags_lemmas_stems_spacy_nltk.pkl').reset_index(drop=True)


In [137]:
%%time
df_manual['Job Description spacy_1grams_original_list'] = df_manual['Job Description spacy_tokenized']
df_manual['Job Description spacy_1grams'] = df_manual['Job Description spacy_tokenized'].progress_apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 20.6 ms, sys: 5.3 ms, total: 25.9 ms
Wall time: 28 ms


In [138]:
%%time
# Spacy bi and trigrams
matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['NOUN', 'VERB'],
    ['VERB', 'NOUN'],
    ['ADJ', 'NOUN'],
    ['ADJ', 'PROPN'],
    # more rules here...
]

trigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

patters_dict = {
    'bigram_patterns': [[{'POS': i} for i in j] for j in bigram_rules],
    'trigram_patterns': [[{'POS': i} for i in j] for j in trigram_rules],
}

ngram_dict = {
    'bigram': 2,
    'trigram': 3,
}

for ngram_name, ngram_num in tqdm.tqdm(ngram_dict.items()):

    matcher.add(f'{ngram_name}_patterns', patters_dict[f'{ngram_name}_patterns'])

    df_manual[f'Job Description spacy_{str(ngram_num)}grams_original_list'] = df_manual['Job Description spacy_sentencized'].progress_apply(
        lambda sentence: 
            [
                '_'.join(ngram_.split())
                for ngram_ in spacy_make_ngrams(sentence, matcher, ngram_name)
            ]
    )

    df_manual[f'Job Description spacy_{str(ngram_num)}grams'] = df_manual['Job Description spacy_sentencized'].progress_apply(
        lambda sentence: 
            [
                tuple(ngram_.split())
                for ngram_ in spacy_make_ngrams(sentence, matcher, ngram_name)
            ]
    )

    df_manual[f'Job Description spacy_{str(ngram_num)}grams_in_sent'] = df_manual['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[f'Job Description spacy_{str(ngram_num)}grams_original_list']
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )

    if f'{ngram_name}_patterns' in matcher:
        matcher.remove(f'{ngram_name}_patterns')
    assert f'{ngram_name}_patterns' not in matcher


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 1min 59s, sys: 1.86 s, total: 2min 1s
Wall time: 2min 7s


In [139]:
%%time
# Spacy Allgrams
df_manual['Job Description spacy_123grams_original_list'] = df_manual['Job Description spacy_tokenized'] + df_manual['Job Description spacy_2grams_original_list'] + df_manual['Job Description spacy_3grams_original_list']
df_manual['Job Description spacy_123grams'] = df_manual['Job Description spacy_1grams'] + df_manual['Job Description spacy_2grams'] + df_manual['Job Description spacy_3grams']
df_manual['Job Description spacy_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description spacy_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


CPU times: user 21.6 s, sys: 260 ms, total: 21.8 s
Wall time: 23.4 s


In [140]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy.csv', index=False)


# Use NLTK to create bi and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [141]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [142]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [143]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy.pkl').reset_index(drop=True)


In [144]:
%%time
df_manual['Job Description nltk_1grams_original_list'] = df_manual['Job Description nltk_tokenized']
df_manual['Job Description nltk_1grams'] = df_manual['Job Description nltk_tokenized'].progress_apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 15.7 ms, sys: 1.91 ms, total: 17.6 ms
Wall time: 22.5 ms


In [145]:
%%time
# NLTK bi and trigrams
ngram_dict = {
    'bigram': 2,
    'trigram': 3
}

for ngram_name, ngram_num in tqdm.tqdm(ngram_dict.items()):

    df_manual[f'Job Description nltk_{str(ngram_num)}grams_original_list'] = df_manual['Job Description nltk_tokenized'].progress_apply(
        lambda tokens:
            list(
                '_'.join(ngram_list)
                for ngram_list in nltk.ngrams(tokens, ngram_num)
            )
    )

    df_manual[f'Job Description nltk_{str(ngram_num)}grams'] = df_manual['Job Description nltk_tokenized'].progress_apply(
        lambda tokens: list(nltk.ngrams(tokens, ngram_num))
    )

    df_manual[f'Job Description nltk_{str(ngram_num)}grams_in_sent'] = df_manual['Job Description spacy_sentencized'].str.lower().replace(
        regex = {
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[f'Job Description nltk_{str(ngram_num)}grams_original_list']
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 3min 10s, sys: 1.67 s, total: 3min 12s
Wall time: 3min 30s


In [146]:
%%time
# NLTK Allgrams
df_manual['Job Description nltk_123grams_original_list'] = (
    df_manual['Job Description nltk_tokenized']
    + df_manual['Job Description nltk_2grams_original_list']
    + df_manual['Job Description nltk_3grams_original_list']
)
df_manual['Job Description nltk_123grams'] = (
    df_manual['Job Description nltk_1grams']
    + df_manual['Job Description nltk_2grams']
    + df_manual['Job Description nltk_3grams']
)
df_manual['Job Description nltk_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description nltk_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


CPU times: user 3min 9s, sys: 1.45 s, total: 3min 11s
Wall time: 3min 20s


In [147]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy_nltk.csv', index=False)


# Use Gensim to create bi and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [148]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [149]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [150]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk.pkl').reset_index(drop=True)


In [151]:
df_manual['Job Description gensim_1grams_original_list'] = df_manual['Job Description gensim_tokenized']
df_manual['Job Description gensim_1grams'] = df_manual['Job Description gensim_tokenized'].progress_apply(
    lambda tokens: [
        tuple(token.split())
        for token in tokens
    ]
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

In [152]:
%%time
# Gensim bi and trigrams
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

# Gensim Bigrams
bigram = Phraser(Phrases(df_manual['Job Description gensim_tokenized'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_manual['Job Description gensim_2grams_original_list_all'] = bigram[df_manual['Job Description gensim_tokenized']]
df_manual['Job Description gensim_2grams_original_list'] = df_manual['Job Description gensim_2grams_original_list_all'].progress_apply(
    lambda ngrams_list: [
        ngram_
        for ngram_ in ngrams_list
        if len(re.findall('[a-zA-Z]*\_[a-zA-Z]*', ngram_)) != 0
    ]
)
df_manual['Job Description gensim_2grams'] = df_manual['Job Description gensim_2grams_original_list'].progress_apply(
    lambda ngrams: [
        tuple(ngram.split('_'))
        for ngram in ngrams
        if '_' in ngram
    ]
)
df_manual['Job Description gensim_2grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .progress_apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_2grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)

# Gensim Trigrams
trigram = Phraser(Phrases(df_manual['Job Description gensim_2grams_original_list_all'], connector_words=ENGLISH_CONNECTOR_WORDS, min_count=1, threshold=1))
df_manual['Job Description gensim_3grams_original_list_all'] = trigram[df_manual['Job Description gensim_2grams_original_list_all']]
df_manual['Job Description gensim_3grams_original_list'] = df_manual['Job Description gensim_3grams_original_list_all'].progress_apply(
    lambda ngrams_list: [
        ngram_
        for ngram_ in ngrams_list
        if len(re.findall('[a-zA-Z]*\_[a-zA-Z]*\_[a-zA-Z]*', ngram_)) != 0
    ]
)
df_manual['Job Description gensim_3grams'] = df_manual['Job Description gensim_3grams_original_list'].progress_apply(
    lambda ngrams: [
        tuple(ngram.split('_'))
        for ngram in ngrams
        if '_' in ngram
    ]
)
df_manual['Job Description gensim_3grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .progress_apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_3grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 16.2 s, sys: 218 ms, total: 16.4 s
Wall time: 17.4 s


In [153]:
%%time
# Gensim Allgrams
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'

df_manual['Job Description gensim_123grams_original_list'] = (
    df_manual['Job Description gensim_tokenized']
    + df_manual['Job Description gensim_2grams_original_list']
    + df_manual['Job Description gensim_3grams_original_list']
)
df_manual['Job Description gensim_123grams'] = (
    df_manual['Job Description gensim_1grams']
    + df_manual['Job Description gensim_2grams']
    + df_manual['Job Description gensim_3grams']
)
df_manual['Job Description gensim_123grams_in_sent'] = (
    df_manual['Job Description spacy_sentencized']
    .str.lower()
    .progress_apply(
        lambda sentence: ' '.join(
            preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
        )
    )
    .replace(
        regex={
            re.escape(' '.join(ngram_.split('_'))): re.escape(ngram_)
            for ngrams_list in df_manual[
                'Job Description gensim_123grams_original_list'
            ]
            for ngram_ in ngrams_list
            if '_' in ngram_
        }
    )
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 17.3 s, sys: 361 ms, total: 17.7 s
Wall time: 22.2 s


In [154]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.csv', index=False)


# Create word frequencies for uni, bi, and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [155]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [156]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [157]:
def get_abs_frequency(row, text_col, ngram_num, embedding_library):

    abs_word_freq = defaultdict(int)
    for word in row[f'Job Description {embedding_library}_{ngram_num}grams_original_list']:
        abs_word_freq[word] += 1

        abs_wtd_df = (
            pd.DataFrame.from_dict(abs_word_freq, orient='index')
            .rename(columns={0: 'abs_word_freq'})
            .sort_values(by=['abs_word_freq'], ascending=False)
            )
        abs_wtd_df.insert(1, 'abs_word_perc', value=abs_wtd_df['abs_word_freq'] / abs_wtd_df['abs_word_freq'].sum())
        abs_wtd_df.insert(2, 'abs_word_perc_cum', abs_wtd_df['abs_word_perc'].cumsum())

        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_freq'] = str(abs_wtd_df['abs_word_freq'].to_dict())
        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc'] = str(abs_wtd_df['abs_word_perc'].to_dict())
        row[f'Job Description {embedding_library}_{ngram_num}grams_abs_word_perc_cum'] = str(abs_wtd_df['abs_word_perc_cum'].to_dict())

    return row


In [158]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_spacy_nltk_gensim.pkl').reset_index(drop=True)


In [159]:
%%time
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, ngram_num in tqdm_product(embedding_libraries_list, ngrams_list):
    df_manual[f'Job Description {embedding_library}_{ngram_num}grams_count'] = df_manual[f'Job Description {embedding_library}_{ngram_num}grams'].apply(lambda x: len(x))
    df_manual = df_manual.progress_apply(lambda row: get_abs_frequency(row=row, text_col='Job Description spacy_tokenized', ngram_num=ngram_num, embedding_library=embedding_library), axis='columns')


  0%|          | 0/12 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

<timed exec>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

<timed exec>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


...

<timed exec>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

<timed exec>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 7min 24s, sys: 35.4 s, total: 7min 59s
Wall time: 8min 25s


In [160]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_frequency.csv', index=False)


# Create BoW dictionary, corpus, and tfidf matrix for uni, bi, and trigrams


### START HERE IF SOURCING FROM df_manual_NGRAMS_FREQUENCY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [161]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [162]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [163]:
def get_corpus_and_dictionary(row, ngram_num, embedding_library):
    
    ngrams_original_list = row[f'Job Description {embedding_library}_{ngram_num}grams_original_list']
    dictionary = Dictionary([ngrams_original_list])
    BoW_corpus = [dictionary.doc2bow(ngrams_original_list)]
    tfidf = TfidfModel(BoW_corpus, smartirs='ntc')
    tfidf_matrix = [tfidf[doc] for doc in BoW_corpus]

    row[f'Job Description {embedding_library}_{ngram_num}grams_dictionary'] = dictionary
    row[f'Job Description {embedding_library}_{ngram_num}grams_BoW_corpus'] = BoW_corpus
    row[f'Job Description {embedding_library}_{ngram_num}grams_tfidf'] = tfidf
    row[f'Job Description {embedding_library}_{ngram_num}grams_tfidf_matrix'] = tfidf_matrix
    
    return row


In [164]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl').reset_index(drop=True)


In [165]:
%%time
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']
for embedding_library, ngram_num in tqdm_product(embedding_libraries_list, ngrams_list):
    df_manual = df_manual.progress_apply(
        lambda row: get_corpus_and_dictionary(
            row=row, ngram_num=ngram_num, embedding_library=embedding_library
        ),
        axis='columns'
    )

assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_frequency.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_BoW.csv', index=False)


  0%|          | 0/12 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

...

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 1min 5s, sys: 6.83 s, total: 1min 12s
Wall time: 1min 21s


In [166]:
df_manual.columns


Index(['% Sector per Workforce', '% per Sector', '% per Social Category', '% per Workforce', 'Age', 'Age_Mixed', 'Age_Num', 'Age_Older', 'Age_Older_% per Sector', 'Age_Older_% per Social Category',
       ...
       'Job Description gensim_2grams_tfidf', 'Job Description gensim_2grams_tfidf_matrix', 'Job Description gensim_3grams_dictionary', 'Job Description gensim_3grams_BoW_corpus', 'Job Description gensim_3grams_tfidf', 'Job Description gensim_3grams_tfidf_matrix', 'Job Description gensim_123grams_dictionary', 'Job Description gensim_123grams_BoW_corpus', 'Job Description gensim_123grams_tfidf', 'Job Description gensim_123grams_tfidf_matrix'], dtype='object', length=202)

In [167]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_ngrams_BoW.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_ngrams_BoW.csv', index=False)


# ATTN: This script should be run AFTER all bi and trigrams (spacy, nltk, and gensim) completed.


# Use spacy and nltk for sentiment scoring


### START HERE IF SOURCING FROM df_manual_NGRAMS_BOW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [168]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [169]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [170]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_ngrams_BoW.pkl').reset_index(drop=True)


In [171]:
%%time
# Spacy sentiment
if 'spacytextblob' not in nlp.pipe_names:
    nlp.add_pipe('spacytextblob')

df_manual['Job Description spacy_sentiment'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: float(nlp(sentence)._.blob.polarity)
    if isinstance(sentence, str) else np.nan
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 26.7 s, sys: 317 ms, total: 27 s
Wall time: 28.1 s


In [172]:
%%time
# NLTK sentiment
df_manual['Job Description nltk_sentiment'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: float(sentim_analyzer.polarity_scores(sentence)['compound'])
    if isinstance(sentence, str) else np.nan
)


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

CPU times: user 572 ms, sys: 16.5 ms, total: 589 ms
Wall time: 665 ms


In [173]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_sentiment_spacy_nltk.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_sentiment_spacy_nltk.csv', index=False)


# ATTN: This script should be run AFTER all sentiment scoring (spacy and nltk) completed.


### START HERE IF SOURCING FROM df_manual_SENTIMENT_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


# Word2Vec and FastText embeddings


In [174]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [175]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [176]:
def build_train_word2vec(df, ngram_number, embedding_library, size = 300, words=None, t = time.time(), cores = multiprocessing.cpu_count()):
    if words is None:
        words = [
            'she',
            'he',
            'support',
            'leader',
            'management',
            'team',
            'business',
            'customer',
            'risk',
            'build',
            'computer',
            'programmer',
        ]
    sentences = df[f'Job Description {embedding_library}_{ngram_number}grams_original_list'].values

    w2v_model = Word2Vec(
        sentences=sentences,
        vector_size=size,
        min_count=0,
        window=2,
        sample=6e-5,
        alpha=0.03,
        min_alpha=0.0007,
        negative=20,
        workers=cores - 1,
        sg = 1,
    )

    w2v_model.build_vocab(sentences, progress_per=10000)
    print(f'Time to train the model for {size}: {round((time.time() - t) / 60, 2)} mins')

    w2v_model.train(
        sentences,
        total_examples=w2v_model.corpus_count,
        epochs=30,
        report_delay=1,
    )

    print(f'Time to build w2v_vocab for {size}: {round((time.time() - t) / 60, 2)} mins')
    w2v_vocab = list(w2v_model.wv.index_to_key)

    print(f'Checking words form list of length {len(words)}')
    print(f'WORDS LIST: {words}')

#     for word in words:
#         print(f'Checking word:\n{word.upper()}:')
#         try:
# #             print(f'Word2Vec {size}: {w2v_model.wv[word]}')
#             print(f'Length of {size} model vobal: {len(w2v_vocab)}')
#             print(f'{size} - Positive most similar to {word}: {w2v_model.wv.most_similar(positive=word, topn=5)}')
#             print(f'{size} - Negative most similar to {word}: {w2v_model.wv.most_similar(negative=word, topn=5)}')

#         except KeyError as e:
#             print(e)

    return w2v_vocab, w2v_model

def word2vec_embeddings(sentences, w2v_vocab, w2v_model, size=300):

    sentences = [word for word in sentences if word in w2v_vocab]

    return (
        np.mean(w2v_model.wv[sentences], axis=0)
        if sentences
        else np.zeros(size)
    )



In [177]:
def build_train_fasttext(df, ngram_number, embedding_library, size = 300, words=None, t = time.time(), cores = multiprocessing.cpu_count()):
    if words is None:
        words = [
            'she',
            'he',
            'support',
            'leader',
            'management',
            'team',
            'business',
            'customer',
            'risk',
            'build',
            'computer',
            'programmer',
        ]
    sentences = df[f'Job Description {embedding_library}_{ngram_number}grams_original_list'].values

    ft_model = FastText(
        sentences=sentences,
        vector_size=size,
        min_count=0,
        window=2,
        sample=6e-5,
        alpha=0.03,
        min_alpha=0.0007,
        negative=20,
        workers=cores - 1,
        sg = 1,
    )

    ft_model.build_vocab(sentences, progress_per=10000)
    print(f'Time to train the model for {size}: {round((time.time() - t) / 60, 2)} mins')

    ft_model.train(
        sentences,
        total_examples=ft_model.corpus_count,
        epochs=30,
        report_delay=1,
    )

    print(f'Time to build vocab for {size}: {round((time.time() - t) / 60, 2)} mins')
    ft_vocab = list(ft_model.wv.index_to_key)

    print(f'Checking words form list of length {len(words)}')
    print(f'WORDS LIST: {words}')

#     for word in words:
#         print(f'Checking word:\n{word.upper()}:')
#         try:
# #             print(f'FastText {size}: {ft_model_300.wv[word]}')
#             print(f'Length of {size} model vobal: {len(ft_vocab)}')
#             print(f'{size} - Positive most similar to {word}: {ft_model.wv.most_similar(positive=word, topn=5)}')
#             print(f'{size} - Negative most similar to {word}: {ft_model.wv.most_similar(negative=word, topn=5)}')

#         except KeyError as e:
#             print(e)

    return ft_vocab, ft_model

def fasttext_embeddings(sentences, ft_vocab, ft_model, size=300):

    sentences = [word for word in sentences if word in ft_vocab]

    return np.mean(ft_model.wv[sentences], axis=0) if sentences else np.zeros(size)


In [178]:
def get_glove(glove_file = f'{llm_path}/gensim/glove/glove.840B.300d.txt'):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf8') as glove:

        for line in glove:
            values = line.split()
            word = values[0]

            with contextlib.suppress(ValueError):
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
    print(f'Found {len(embeddings_index)} word vectors.')

    return embeddings_index


In [179]:
def sent2vec(sentences, embeddings_index=None, external_glove=True, extra_preprocessing_enabled=False):

    if external_glove is False and embeddings_index is None:
        embeddings_index= get_glove()

    if extra_preprocessing_enabled is False:
        words = sentences

    elif extra_preprocessing_enabled is True:
        stop_words = set(sw.words('english'))
        words = str(sentences).lower()
        words = word_tokenize(words)
        words = [w for w in words if (w not in stop_words) and (w.isalpha())]

    M = []

    try:
        for w in words:
            try:
                M.append(embeddings_index[w])
            except Exception:
                continue

        M = np.array(M)
        v = M.sum(axis='index')
        return np.zeros(300) if type(v) != np.ndarray else v / np.sqrt((v ** 2).sum())

    except Exception:
        return np.zeros(300)


In [180]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_sentiment_spacy_nltk.pkl').reset_index(drop=True)


In [181]:
embedding_models_dict = {
    'w2v': [build_train_word2vec, word2vec_embeddings, Word2Vec],
    'ft': [build_train_fasttext, fasttext_embeddings, FastText],
}


In [182]:
%%time
# Make embeddings
ngrams_list=[1, 2, 3, 123]
embedding_libraries_list = ['spacy', 'nltk', 'gensim']

for embedding_library, ngram_number in tqdm_product(embedding_libraries_list, ngrams_list):
    print(f'Building {embedding_library}_{ngram_number}grams model and vocabulary.')

    for embed_model_name, embed_func_list in embedding_models_dict.items():

        build_train_func, embed_func, model_loader = embed_func_list
        print(f'Building {embed_model_name} from {embed_func.__name__} function.')

        vocab, model = build_train_func(
            df=df_manual,
            ngram_number=ngram_number,
            embedding_library=embedding_library,
        )

        print(f'Getting {embed_model_name} embeddings.')

        df_manual[
            f'Job Description {embedding_library}_{ngram_number}grams_mean_{embed_model_name}_embeddings'
        ] = df_manual[
            f'Job Description {embedding_library}_{ngram_number}grams_original_list'
        ].progress_apply(
            lambda sentences: embed_func(sentences, vocab, model)
        )
        model.save(f'{data_dir}embeddings models/{embedding_library}_{ngram_number}grams_{embed_model_name}_model.model')

    # Sent2Vec
    print('Getting sent2vec embeddings.')
    embeddings_index = get_glove()
    df_manual[f'Job Description {embedding_library}_{ngram_number}grams_sent2vec_embeddings'] = df_manual[f'Job Description {embedding_library}_{ngram_number}grams'].progress_apply(lambda sentences: sent2vec(sentences, embeddings_index=embeddings_index, external_glove=True, extra_preprocessing_enabled=False))
    print('Done getting sent2vec embeddings.')


  0%|          | 0/12 [00:00<?, ?it/s]

Building spacy_1grams model and vocabulary.
Building w2v from word2vec_embeddings function.


Time to train the model for 300: 0.08 mins
Time to build w2v_vocab for 300: 0.13 mins
Checking words form list of length 12
WORDS LIST: ['she', 'he', 'support', 'leader', 'management', 'team', 'business', 'customer', 'risk', 'build', 'computer', 'programmer']
Getting w2v embeddings.


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

Building ft from fasttext_embeddings function.


...

Time to train the model for 300: 28.41 mins
Time to build vocab for 300: 28.57 mins
Checking words form list of length 12
WORDS LIST: ['she', 'he', 'support', 'leader', 'management', 'team', 'business', 'customer', 'risk', 'build', 'computer', 'programmer']
Getting ft embeddings.


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

Getting sent2vec embeddings.
Found 2195885 word vectors.


progress-bar:   0%|          | 0/5365 [00:00<?, ?it/s]

Done getting sent2vec embeddings.
CPU times: user 30min 54s, sys: 2min 53s, total: 33min 47s
Wall time: 30min 3s


In [183]:
assert len(df_manual) > 0 and isinstance(df_manual, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_manual)}'
df_manual.to_pickle(f'{df_save_dir}df_manual_for_trainning.pkl')
df_manual.to_csv(f'{df_save_dir}df_manual_for_trainning.csv', index=False)


In [188]:
print(f'Saving df_manual length {len(df_manual)} to txt file.')
with open(f'{data_dir}df_manual_len.txt', 'w') as f:
    f.write(str(len(df_manual)))

Saving df_manual length 5365 to txt file.
